In [5]:
#-----------------------------------------------------------------------------------------------------------------------
# Reading data from sqlite
#-----------------------------------------------------------------------------------------------------------------------
import sqlite3
import pandas as pd

# #Establishing connection with sqlite database
conn = sqlite3.connect('../Sensors.db')
curs = conn.cursor()

df = pd.read_sql_query("SELECT * FROM BME_DATA", conn)
df.drop('id', axis=1, inplace=True)
df.rename(columns={'TIME_STAMP': 'TimeStamp',
                   'TEMPERATURE': 'Temperature',
                   'GAS':'Gas',
                   'HUMIDITY':'Humidity',
                   'PRESSURE':'Pressure',
                   'ALTITUDE':'Altitude'
                   }, inplace=True)
df.head()

df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%Y-%m-%d %H:%M:%S.%f')
df = df.resample('H', on='TimeStamp').agg({'Temperature':'mean', 'Gas':'mean','Humidity':'mean','Pressure':'mean','Altitude':'mean'})
df.reset_index(inplace = True)
df.head()

,TimeStamp,Temperature,Gas,Humidity,Pressure,Altitude
0,2021-01-12 19:00:00,20.673344,355355.546198,40.131398,984.801881,240.378577
1,2021-01-12 20:00:00,19.789287,541956.520300,40.835244,984.766667,239.895394
2,2021-01-12 21:00:00,19.545680,579178.531633,40.634626,984.751667,239.959150
3,2021-01-12 22:00:00,19.219509,577062.787176,40.569236,984.820396,239.426194
4,2021-01-12 23:00:00,19.012534,578171.792575,39.787807,984.439782,242.660082


In [6]:
#-----------------------------------------------------------------------------------------------------------------------
# Pushing data into Auzre mysql
#-----------------------------------------------------------------------------------------------------------------------
# Importing Dependencies
import mysql.connector
import pandas as pd
import json
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table
from mysql.connector import Error

with open("../parameters/config.json") as config:
    param = json.load(config)

# Connecting to data warehouse
try:
    engine = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                      format(param['MyDemoServer'][0]['user'],
                                             param['MyDemoServer'][0]['password'],
                                             param['MyDemoServer'][0]['host'],
                                             param['MyDemoServer'][0]['database']), echo=False)

    # Cleaning the data from existing tables MetricValues and Metrics
    Epi_con = engine.connect()
    if Epi_con.connect():
        try:
            #Epi_con.execute("""TRUNCATE TABLE Sessions""")
            df.to_sql('sensordata', con=Epi_con, if_exists='append',chunksize=1000, index=False)
            Epi_con.close()
            engine.dispose()
        except OSError as e:
            print(e)
except OSError as e:
    print(e)


c:\users\slavo\pycharmprojects\jetsonnano_development\venv\lib\site-packages\pandas\io\sql.py:1685: UserWarning: The provided table name 'SensorData' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
